## Installing Intel Optimized Libraries from Intel AI Analytic Toolkit(Pytorch and Tensorflow)

In [ ]:
# !pip install torch==1.13.1+cpu torchvision==0.14.1+cpu –f https://download.pytorch.org/whl/torch_stable.html
# !pip install intel_extension_for_pytorch==1.13.100 -f https://developer.intel.com/ipex-whl-stable-cpu
# !pip install intel-tensorflow

## Importing Necessary Packages

In [ ]:
import transformers
import os
import io
import pyttsx3
from diffusers import StableDiffusionPipeline
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers.utils import load_image
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from PIL import Image
import intel_extension_for_pytorch as ipex


if("result" not in os.listdir(os.getcwd())):
    os.makedirs("result")

## Loading Text to Image Model

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
#if ipex installed run this or run the line below it
text_to_image_pipeline = ipex.optimize(StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32))
# text_to_image_pipeline = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)

## Function to generate story based on user input


In [ ]:
def generate_story_extension(user_input, tokenizer, model_name, temperature=0.85, max_new_tokens=1000):
 
  generator = pipeline("text-generation", model=model_name, tokenizer=tokenizer)
  story_extension = generator(user_input, max_new_tokens=max_new_tokens, num_return_sequences=1, temperature=temperature)[0]['generated_text'][len(user_input)+1:]

  return story_extension.strip()

## Loading Text to Text model and Tokeniser for the same

In [ ]:
t1 = AutoTokenizer.from_pretrained("Intel/neural-chat-7b-v3-3")
#if ipex installed run this or run the line below it
m1= ipex.optimize(AutoModelForCausalLM.from_pretrained("Intel/neural-chat-7b-v3-3"))
# m1= AutoModelForCausalLM.from_pretrained("Intel/neural-chat-7b-v3-3")

## Defining start story function and further modification according to user prompt

In [ ]:
def start_story():
  list_of_response = []
  updated_story = ""
  base_story = ""

  characters = input("Enter the name of characters along with their descriptions \n")
  plot = input("Describe the plot in brief \n")
  theme = input("Provide a theme of the story along with the setting \n")



  user_input = f"Theme and setting: {theme} plot: {plot} characters: {characters}"

  prompt = f"generate a complete story of at least 800 words based on input given by the user: \n {user_input} "
  base_story = generate_story_extension(prompt, t1, model_name="Intel/neural-chat-7b-v3-3", temperature=0.85, max_new_tokens=1000)
  updated_story = base_story
  narrative_finished = False
  print("base story starts:",updated_story)
  while not narrative_finished :
      user_changed_input = input("If you are satisfied with the story, please type end or type out the changes you want in the form of a simple prompt")
      if user_changed_input.lower()=="end" :
        narrative_finished = True
        break
      prompt_change = f"generate a new story from scratch of at least 800 words with referene to the previously generated story and based on changes instructed by user:- \n {user_changed_input} "
      updated_story =  generate_story_extension(prompt_change, t1, model_name="Intel/neural-chat-7b-v3-3", temperature=0.85, max_new_tokens=1000)
      print(updated_story)
  return updated_story

## Calling start story function and dividing the story into certain number of parts which will be input into the Text to Image model

In [ ]:
story=start_story()
prompt_for_img_gen = f"Based on the story:{story} add \n delimiters to separate the story into at least 10  pivotal parts(each part represent a different pivotal chapter of story) and  at max 20 parts, where each pivotal part gives an illustrative desciption of 30 to 40 words about that part such that an image can be generated from the part and can be fed into a text to image model to show progressive story, try involving new setting or new characters in each part"
prompt_corpus = generate_story_extension(prompt_for_img_gen, t1, model_name="Intel/neural-chat-7b-v3-3", temperature=0.85, max_new_tokens=1000)
p = "Generate an immersive visual representation capturing a pivotal scene from the story, featuring the central character(s) in a vivid setting that encapsulates the thematic essence of the narrative."
print(prompt_corpus)
li=prompt_corpus.split("\n")
story_sequence=[]
for x in li:
  if 'Part' in x:
    story_sequence.append(x)

base_images = []
for i in range(len(story_sequence)):
    if(i>=1):
      image = text_to_image_pipeline(story_sequence[i-1]+story_sequence[i]).images[0]  # Get the image tensor
    else:
      image = text_to_image_pipeline(story_sequence[i]).images[0]  # Get the image tensor
    base_images.append(image)




## Saving images in the result folder

In [ ]:
save_folder="./result"

for i, image in enumerate(base_images):

    file_path = os.path.join(save_folder, f"base_image_{i}.png")
    
    image.save(file_path)

## Displaying images and playing audio simultaneously

In [ ]:

for i,image_path in enumerate(sorted(os.listdir(save_folder))):
    engine = pyttsx3.init()
    engine.setProperty('rate', 180)  
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    if image_path.endswith(".png"):  
        full_path = os.path.join(save_folder, image_path)
        img = Image.open(full_path) 
        img.show()
        text = story_sequence[i]
        engine.say(text)
        engine.runAndWait()

# Tried to implement image to image model but didnt find any improvement in results so ignored it in the final result.(code may not work)

In [ ]:
# def refine_image_via_api(base_image, refinement_prompt):
#     image_to_image_pipeline = pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
#     refined_image_data =image_to_image_pipeline( refinement_prompt , base_image )   # Replace with actual API request
#     refined_image = Image.open(io.BytesIO(refined_image_data))
#     return refined_image

# refined_images = []
# for i, base_image in enumerate(base_images):
#     context = story_sequence[:i+1]  # Context includes previous story scenes
#     context_prompt = " ".join(context)
#     refinement_prompt = f"Refine the image to depict: {context_prompt}"

#     refined_image = refine_image_via_api(base_image, refinement_prompt)
#     refined_images.append(refined_image)

